In [1]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from utils.utils import pretty_lat, pretty_lon, ProgressStatus, plot_n_steps_of_area_from_nc_file
from tempfile import TemporaryDirectory

In [2]:
from station.data_submission import DataSubmission
from station.station import StationData

from train_station_twin.training_executer import TrainingExecuter
from infilling.evaluation_executer import EvaluatuionExecuter
from infilling.infilling_writer import InfillingWriter

from era5.era5_for_station import DownloadEra5ForStation, DownloadEra5ForStationGaps
from era5.era5_download_hook import Era5DownloadHook


In [3]:
marshall_data = DataSubmission("Marshall_2017-2023")
marshall_data.measurement_dir_path = "./measurements/Marshall/"

marshall_station = StationData(
    name=marshall_data.name,
    folder_path=marshall_data.measurement_dir_path
)

Extracting Marshall_2017-2023...


100%|██████████| 2465/2465 [02:24<00:00, 17.02it/s]


In [4]:
marshall_station.df

,vis_light,tas,ir_light,uv_light
datetime,,,,
2017-03-01 05:00:00,NaN,277.316667,NaN,NaN
2017-03-01 18:00:00,NaN,277.786364,NaN,NaN
2017-03-01 19:00:00,NaN,278.873333,NaN,NaN
2017-03-01 20:00:00,NaN,279.076667,NaN,NaN
2017-03-01 21:00:00,NaN,277.528333,NaN,NaN
...,...,...,...,...
2023-05-18 09:00:00,NaN,283.781818,NaN,NaN
2023-06-12 05:00:00,NaN,285.291667,NaN,NaN
2023-06-12 06:00:00,NaN,285.044545,NaN,NaN


In [5]:

from utils.utils import plot_measurements_df
plot_measurements_df(marshall_station.df)

NameError: name 'plt' is not defined

**Plot a few random timesteps of the weather station:**

In [ ]:
random_timesteps = marshall_station.df.index

In [ ]:

training = TrainingExecuter(station=marshall_station, ProgressStatus())

output_model_path = training.execute()
marshall_data.add_model(output_model_path)



SyntaxError: positional argument follows keyword argument (804761686.py, line 1)

In [ ]:
barbados_station = StationData("Barbados", "./measurements/Barbados/")
vienna_station = StationData("Vienna", "./measurements/Vienna/")